In [ ]:
import io
import napari
import numpy as np
import time, os, sys
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
from skimage.io import imread
from deepcell.applications import CytoplasmSegmentation
from deepcell.applications import NuclearSegmentation
from deepcell.applications import Mesmer
from deepcell.applications import CellTracking
from deepcell import io_utils
from tqdm import tqdm

In [ ]:
# Load the image
img = imread('dataset/images_sophie/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_R1_24_C0.tif')

In [ ]:
# Create the application
app = CytoplasmSegmentation()

In [ ]:
# Expand image dimensions to rank 4
#img = np.expand_dims(img, axis=-1)
img_tmp = img
print(img.shape)
img_tmp = np.expand_dims(img_tmp, axis=3)
print(img_tmp.shape)
# create the labeled image
labeled_image = app.predict(img_tmp)
labeled_image = np.squeeze(labeled_image, axis=3)

In [ ]:

#viewer = napari.view_labels(labeled_image, name='labeled_image')
def plot_image(path_to_image) :
    img = imread(path_to_image)
    img = img.squeeze()
    plt.figure(figsize=[4,4])
    plt.imshow(img[25])
    plt.axis('off')
    plt.show()


def plot_image_with_napari(path_to_image) :
    img = imread(path_to_image)
    print(img.shape)
    viewer = napari.view_labels(img, name='image')

def plot_both_original_and_labeled_images(path_to_original_image, path_to_labeled_image) :
    img         = imread(path_to_original_image)
    labeled_img = imread(path_to_labeled_image)
    print(img.shape)
    viewer = napari.view_labels(labeled_image, name='labeled image')

In [ ]:
#viewer = napari.view_labels(labeled_image, name='labeled_image')
#viewer.add_image(img, name='original_image')

In [ ]:
def segment_images_with_deepcell(path_to_images, model, path_to_labeled_images) :
    images = os.listdir(path_to_images)
    for i in tqdm(range(len(images))) :
        image = images[i]
        img = imread(path_to_images+'/'+image)
        img = np.expand_dims(img, axis=3)
        labeled_image = model.predict(img)
        labeled_image = np.squeeze(labeled_image, axis=3)
        print(labeled_image.shape)
        io_utils.imsave(path_to_labeled_images+'/'+image, labeled_image)

segment_images_with_deepcell('dataset/images_sophie', app, 'dataset/labeled_images')

In [ ]:
#Suppresion du bruit
def abs_path(root, listdir_):
    """
    absolute path
    add root to the beginning of each path in listdir
    """
    listdir = listdir_.copy()
    for i in range(len(listdir)):
        listdir[i] = root + '/' + listdir[i]
    return listdir

def abs_listdir(path):
    """
    absolute path
    read all the path of files stored in 'path' 
    and add root to the beginning of each path in listdir
    """
    return abs_path(path, os.listdir(path))

def remove_begin(pattern, listdir_):
    """
    pattern remover
    remove the 'pattern' at the beginning of each files in 'listdir_'
    """
    listdir = listdir_.copy()
    for i in range(len(listdir)):
        listdir[i] = listdir[i][str.find(listdir[i],pattern):]
    return listdir

In [ ]:
# remove unwanted volumes from images
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff

out_path='dataset/labeled_images'
img_out_path='dataset/images_sophie'

fname_msk = os.listdir(out_path)
abs_msk = abs_path(out_path,fname_msk)
abs_img = abs_path(img_out_path,fname_msk)

def tif_read_meta(tif_path, display=False):
    """
    read the metadata of a tif file and stores them in a python dict.
    if there is a 'ImageDescription' tag, it transforms it as a dictionary
    """
    meta = {}
    with tiff.TiffFile(tif_path) as tif:
        for page in tif.pages:
            for tag in page.tags:
                tag_name, tag_value = tag.name, tag.value
                if display: print(tag.name, tag.code, tag.dtype, tag.count, tag.value)

                # below; fix storage problem for ImageDescription tag
                if tag_name == 'ImageDescription': 
                    list_desc = tag_value.split('\n')
                    dict_desc = {}
                    for idx, elm in enumerate(list_desc):
                        split = elm.split('=')
#                         if len(split) > 1:
#                             dict_desc[split[0]] = split[1]
                    meta[tag_name] = dict_desc
                else:
                    meta[tag_name] = tag_value
            break # just check the first image
    return meta

def tif_write_meta(data,meta,out_path):
    """
    write data and metadata in 'out_path'
    """
    out_meta = {
#         'spacing':float(meta['ImageDescription']['spacing']),
#         'unit':meta['ImageDescription']['unit'],
        'axes':'ZYX',
    }
    
    extratags = []
    
    tiff.imwrite(
        out_path,
        data=data,
        resolution=(meta['XResolution'],meta['YResolution']),
        metadata=out_meta,
        extratags=extratags,
#         imagej=True,
    )

def tif_copy_meta(in_path1, in_path2, out_path):
    """
    store (metadata of in_path1 + data of in_path2) in out_path
    """
    in_meta = tif_read_meta(in_path1)
    data = tiff.imread(in_path2)
    tif_write_meta(data, in_meta, out_path)

def tif_copy_meta_numpy(in_path1, data, out_path):
    """
    store (metadata of in_path1 + data of in_path2) in out_path
    """
    in_meta = tif_read_meta(in_path1)
    tif_write_meta(data, in_meta, out_path)

# volume of a connected component
def connec_volume(connec, idx):
    """
    returns the volume in pixel of the connected component of index 'idx' form the image called 'connec'
    """
    return np.sum(connec == idx)

# print the volume of all the connected components in pixel
def connec_volume_all(connec):
    nb_connec = len(np.unique(connec))
    l_connec = []
    for idx, elm in enumerate(np.unique(connec)[1:]): # remove the bg
        l_connec += [connec_volume(connec, elm)]
    return l_connec

def keep_big_connec(abs_msk):
    """
    keep the biggest connected component in the abs_msk list of path
    and store it back inplace
    """
    for i in range(len(abs_msk)):
        msk = imread(abs_msk[i])

        # connected components
        connec = label(msk)
        nb_connec = len(np.unique(connec))

        if nb_connec > 2:

            print("number of connected components: {}".format(nb_connec))
#             plt.imshow(msk[30])
#             plt.show()

            l_connec = connec_volume_all(connec)
            print(l_connec) # list of all connected component volumes (except of the bg)


            # keep only the biggest
            connec_clean = np.zeros_like(connec)
            connec_clean += (connec==(np.argmax(l_connec)+1)).astype(int)*255

            # store back the "clean" mask
            msk = connec_clean.copy().astype(np.uint8)
#             meta_path = '/home/gumougeot/all/data/3d_nucleus/all_manual/images_manual/'
#             meta_path += abs_msk[i].split('/')[-1]
#             print(meta_path)
#             tif_copy_meta_numpy(meta_path,msk,abs_msk[i])
            imsave(abs_msk[i], msk)
#             print(type(msk), msk.shape, np.min(msk), np.max(msk))

#             plt.imshow(msk[30])
#             plt.show()

keep_big_connec(abs_msk)

In [ ]:
def standardize_one_images(image) :
    flag = "OK"
    img = imread(f"dataset/labeled_images/{image}")
    for i in range(img.shape[0]) :
        for j in range(img.shape[1]) :
            for k in range(img.shape[2]) :
                if(img[i][j][k] != 0 and img[i][j][k] != 255) :
                    flag = "warning"
                    img[i][j][k] = 255
    if(flag == "warning"): 
        io_utils.imsave(f"dataset/labeled_images/{image}", img)
    print(flag)

def standardize_all_images(path) :
    images = os.listdir(path)
    for image in images :
        standardize_one_images(image)

standardize_all_images("dataset/labeled_images")


In [ ]:
plot_image('dataset/labeled_images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_M2_0_C0.tif')
plot_image_with_napari('dataset/labeled_images/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_M2_0_C0.tif')